In [ ]:
import numpy as np
import pandas as pd
import googlemaps
from os import path

In [ ]:
sourceFile = 'data/utf8_areeVerdiMilano.csv'
targetSuffix = '_geoloc'

# derive target filename
targetFile = targetSuffix.join(list(path.os.path.splitext(sourceFile)))
origData = pd.read_csv(sourceFile, sep=';')

In [ ]:

## build query addresses
queryAddresses = origData.iloc[:,3].values + ', Milano'

In [ ]:
gmaps = googlemaps.Client(key=key)

nLimit = 1500

# Geocoding an address
locationsFull = pd.Series.from_array([{}]*len(queryAddresses))
# hardcoded coordinates format
coordinates = pd.DataFrame([[np.nan, np.nan]]*len(queryAddresses), columns=['Long', 'Lat'])
multipleResults = pd.Series.from_array([]*len(queryAddresses))

for (i, address) in enumerate(queryAddresses): 
    if i < nLimit:
        continue
        #check that this is never run
        error()
    geocode_result = gmaps.geocode(address, components={'location':'Milano'})
    if geocode_result:
        assert isinstance(geocode_result, list) #check type
        if len(geocode_result) > 1: # multiple results, report first and
            multipleResults[i] = geocode_result
        
        # take first output
        locationsFull[i] = geocode_result[0]
            
        # parse coordinates
        thisLocation = locationsFull[i]['geometry']['location']
        coordinates['Long'][i] = thisLocation['lng']
        coordinates['Lat'][i] = thisLocation['lat']
        print(coordinates.iloc[i,:])
        
    print(i)


In [ ]:
# save full responses
import pickle
pickle_out = open(targetFile + '_results.pickle',"wb")
pickle.dump({'fullLoc': locationsFull, 'extraResults': multipleResults}, pickle_out)
pickle_out.close()


In [ ]:
geolocData = pd.concat([origData, coordinates], axis=1)
geolocData.to_csv(targetFile, sep=';', decimal=',')
#coordinates.to_csv('coordinateAreeVerdi.csv', sep=';', decimal=',')

In [ ]:
geolocData